In [1]:
# -*- coding: utf-8 -*-
import arcpy
from arcpy import env
from arcpy.sa import *
import os

# 输入输出路径设置
input_folder = r"E:\rural rooftop and courtyard\calculate_GIS\prediction_2_23"
# 修改为最终合并后的要素类输出路径
output_merged_fc = r"E:\rural rooftop and courtyard\calculate_GIS\calculate_2_23\calculate_2_23.gdb\Merged_Polygons"
target_coordinate_system = arcpy.SpatialReference(32650)  # WGS 1984 UTM Zone 50N 的投影坐标系代码

# 处理参数设置
target_values = [1, 2, 3]  # 目标像素值
number_cells = 2           # 收缩/扩展次数
resample_method = "NEAREST" # 重采样方法

# 初始化环境设置
env.workspace = input_folder
env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# 创建或指定文件地理数据库作为临时存储
temp_gdb_path = r"E:\rural rooftop and courtyard\calculate_GIS\calculate\RGB_Poly.gdb"
if not arcpy.Exists(temp_gdb_path):
    arcpy.management.CreateFileGDB(os.path.dirname(temp_gdb_path), os.path.basename(temp_gdb_path))

# 用于存储每次栅格转面生成的要素类路径
all_polygons = []

def process_raster(raster):
    """处理单个栅格的工作流程"""
    try:
        # 获取完整的栅格路径
        raster_path = os.path.join(env.workspace, raster)
        print(f"正在处理栅格文件: {raster_path}")

        # 检查输入栅格文件是否存在
        if not arcpy.Exists(raster_path):
            print(f"输入栅格文件 {raster_path} 不存在，请检查路径。")
            return False

        # 生成临时文件名称，去掉扩展名
        raster_name = os.path.splitext(os.path.basename(raster_path))[0]
        temp_projected = os.path.join(temp_gdb_path, f"proj_{raster_name}")
        temp_polygon = os.path.join(temp_gdb_path, f"poly_{raster_name}")

        # 1. 投影坐标系变换
        print("开始进行投影坐标系变换...")
        arcpy.management.ProjectRaster(
            in_raster=raster_path,
            out_raster=temp_projected,
            out_coor_system=target_coordinate_system,
            resampling_type=resample_method
        )
        if not arcpy.Exists(temp_projected):
            print(f"投影坐标系变换失败，未能生成 {temp_projected}")
            return False
        print("投影坐标系变换完成。")

        # 2. 创建掩膜（排除背景值0）
        print("开始创建掩膜...")
        masked_raster = SetNull(temp_projected, temp_projected, "VALUE = 0")
        print("掩膜创建完成。")

        # 3. 形态学处理
        print("开始进行形态学处理...")
        shrink_raster = Shrink(masked_raster, number_cells, target_values)
        final_raster = Expand(shrink_raster, number_cells, target_values)
        final_raster = Int(final_raster)  # 确保为整数类型
        print("形态学处理完成。")

        # 4. 栅格转面
        print("开始进行栅格转面操作...")
        arcpy.conversion.RasterToPolygon(
            in_raster=final_raster,
            out_polygon_features=temp_polygon,
            simplify="SIMPLIFY",
            raster_field="VALUE",
            create_multipart_features="SINGLE_OUTER_PART"
        )
        if not arcpy.Exists(temp_polygon):
            print(f"栅格转面操作失败，未能生成 {temp_polygon}")
            return False
        print("栅格转面操作完成。")

        # 将生成的要素类路径添加到列表中
        all_polygons.append(temp_polygon)

        return True

    except arcpy.ExecuteError:
        print(arcpy.GetMessages(2))  # 输出详细的错误信息
        return False
    except Exception as e:
        print(f"处理 {raster_path} 失败: {str(e)}")
        return False

# 遍历处理所有栅格
rasters = arcpy.ListRasters()
total = len(rasters)
success = 0

for i, raster in enumerate(rasters, 1):
    print(f"正在处理 ({i}/{total}): {raster}")
    if process_raster(raster):
        success += 1

# 检查是否有要素类需要合并
if all_polygons:
    print(f"找到 {len(all_polygons)} 个要素类，开始合并...")
    # 使用Merge工具合并要素类
    arcpy.management.Merge(all_polygons, output_merged_fc)
    print(f"合并完成！结果已保存到：{output_merged_fc}")
else:
    print("未找到需要合并的要素类！")

# 清理和报告结果
arcpy.CheckInExtension("Spatial")
arcpy.management.Delete("in_memory")
print(f"处理完成！成功处理 {success}/{total} 个栅格。")

正在处理 (1/67): 10_predictiononeband.TIF
正在处理栅格文件: E:\rural rooftop and courtyard\calculate_GIS\prediction_2_23\10_predictiononeband.TIF
开始进行投影坐标系变换...
投影坐标系变换完成。
开始创建掩膜...
掩膜创建完成。
开始进行形态学处理...
形态学处理完成。
开始进行栅格转面操作...
栅格转面操作完成。
正在处理 (2/67): 11_predictiononeband.TIF
正在处理栅格文件: E:\rural rooftop and courtyard\calculate_GIS\prediction_2_23\11_predictiononeband.TIF
开始进行投影坐标系变换...
投影坐标系变换完成。
开始创建掩膜...
掩膜创建完成。
开始进行形态学处理...
形态学处理完成。
开始进行栅格转面操作...
栅格转面操作完成。
正在处理 (3/67): 12_predictiononeband.TIF
正在处理栅格文件: E:\rural rooftop and courtyard\calculate_GIS\prediction_2_23\12_predictiononeband.TIF
开始进行投影坐标系变换...
投影坐标系变换完成。
开始创建掩膜...
掩膜创建完成。
开始进行形态学处理...
形态学处理完成。
开始进行栅格转面操作...
栅格转面操作完成。
正在处理 (4/67): 13_predictiononeband.TIF
正在处理栅格文件: E:\rural rooftop and courtyard\calculate_GIS\prediction_2_23\13_predictiononeband.TIF
开始进行投影坐标系变换...
投影坐标系变换完成。
开始创建掩膜...
掩膜创建完成。
开始进行形态学处理...
形态学处理完成。
开始进行栅格转面操作...
栅格转面操作完成。
正在处理 (5/67): 14_predictiononeband.TIF
正在处理栅格文件: E:\rural rooftop and courtyard\calculate_GIS\prediction_